# Permutations of a Simple Circuit

This notebook walks through how to utilize the core semantics of SysML v2 to generate alternative circuits as inputs to an OpenMDAO solution of these circuits. 

## Background

The M1 user model in SysML v2 is meant to be a set of constraints and rules under which legal instances can be created. Those instances should be taken as alternative produced systems and they can be analyzed in that way.

## Libraries Load-Up

Load up PyMBE and its various libraries.

In [1]:
NUM_INTERPRETATIONS = 50
NUM_RESISTORS = (2, 4)
NUM_DIODES = (0, 1)
NUM_CONNECTORS = (5, 10)

RUN_BASELINE = True

In [2]:
from pathlib import Path
import networkx as nx
import matplotlib as plt

import pymbe.api as pm

from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.interpretation import repack_instance_dictionaries
from pymbe.interpretation.interp_playbooks import (
    build_expression_sequence_templates,
    build_sequence_templates,
    random_generator_playbook,
    random_generator_phase_1_multiplicities,
)
from pymbe.interpretation.results import *
from pymbe.label import get_label_for_id
from pymbe.query.metamodel_navigator import feature_multiplicity
from pymbe.query.query import (
    roll_up_multiplicity,
    roll_up_upper_multiplicity,
    roll_up_multiplicity_for_type,
    get_types_for_feature,
    get_features_typed_by_type,
)
from pymbe.local.stablization import build_stable_id_lookups

## Load Up Model

Read the model from the local JSON file.

In [3]:
circuit_file = Path(pm.__file__).parent / "../../tests/fixtures/Circuit Builder.json"

circuit_model = pm.Model.load_from_file(circuit_file)
circuit_lpg = SysML2LabeledPropertyGraph(model=circuit_model)
[id_to_circuit_name_lookup, circuit_name_to_id_lookup] = build_stable_id_lookups(circuit_lpg)

circuit_lpg.model.MAX_MULTIPLICITY = 100

These edge types are not in the graph: {'ImpliedReferentFeed', 'ImpliedParameterFeedforward', 'ImpliedPathArgumentFeedforward'}.
Duplicate name found for: 'Model::Occurrences::Occurrence::04119598-4507-4302-93aa-74c848f3c9b2 «Invariant»::None => Null Result::test::== => Null Result::s.endShot => Null Result::mapper <<FeatureReferenceExpression>>'
Duplicate name found for: 'Model::SequenceFunctions::includesOnly::&& => Null Result::None => Null Result::FRE.seq1 <<FeatureReferenceExpression>>'
Duplicate name found for: 'Model::SequenceFunctions::equals::&& => Null Result::secondValue: forAll::test::== => Null Result::[ => Null Result <<OperatorExpression>>'
Duplicate name found for: 'Model::SequenceFunctions::equals::&& => Null Result::secondValue::size => Null Result <<InvocationExpression>>'
Duplicate name found for: 'Model::StatePerformance::triggerDuring: HappensDuring::a633cff3-8f09-40f6-9b8e-c8f9fceb8a0d «Invariant»::| => Null Result::! => Null Result <<OperatorExpression>>'
Duplic

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\edge_generators.py:149: UserWarning: Found 0 Expression Results and 1 Return Parameter Members!
  warn(
c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\edge_generators.py:149: UserWarning: Found 0 Expression Results and 2 Return Parameter Members!
  warn(
c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:183: UserWarning: Could not find an implied edge generator for 'ImpliedPathArgumentFeedforward'
  warn(f"Could not find an implied edge generator for '{implied_edge_type}'")
c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:183: UserWarning: Could not find an implied edge generator for 'ImpliedReferentFeed'
  warn(f"Could not find an implied edge generator for '{implied_edge_type}'")


## Explore Contents of Model with M1 in Memory

Use the M1 memory objects to see what is in the current model, starting with the main packages.

In [4]:
circuit_model.packages

(<Objects «Package»>,
 <ControlFunctions «Package»>,
 <States «Package»>,
 <Transfers «Package»>,
 <Base «Package»>,
 <ScalarFunctions «Package»>,
 <ControlPerformances «Package»>,
 <TransitionPerformances «Package»>,
 <SequenceFunctions «Package»>,
 <Circuit Builder «Package»>,
 <Parts «Package»>,
 <ScalarValues «Package»>,
 <Links «Package»>,
 <Ports «Package»>,
 <Items «Package»>,
 <Connections «Package»>,
 <Performances «Package»>,
 <Occurrences «Package»>,
 <Constraints «Package»>,
 <StatePerformances «Package»>,
 <Actions «Package»>)

In [5]:
circuit_model.ownedElement["Circuit Builder"].ownedElement

[<Pin «PortDefinition»>,
 <Two Pin «PartDefinition»>,
 <ConnectionDefinition([<Anything «Classifier»>] ←→ [<Anything «Classifier»>])>,
 <Resistor «PartDefinition»>,
 <Diode «PartDefinition»>,
 <EMF «PartDefinition»>,
 <Circuit «PartDefinition»>]

In [6]:
circuit_def = circuit_model.ownedElement["Circuit Builder"].ownedElement["Circuit"]

### Circuit and its Features

Here is the circuit and its features, both parts and used connections.

In [7]:
circuit_def.relationships

{'throughFeatureMembership': [<Motive Force «PartUsage»>,
  <Circuit Diode «PartUsage»>,
  <ConnectionUsage([<01256e61-468b-401b-9e89-3f23a2a10f1f «Feature»>] ←→ [<f869a003-9654-404e-9360-f2397b8a16d5 «Feature»>])>,
  <Component «PartUsage»>,
  <Circuit Resistor «PartUsage»>],
 'reverseMembership': [<Circuit Builder «Package»>]}

In [8]:
circuit_def.ownedMember

[<Component «PartUsage»>,
 <Circuit Resistor «PartUsage»>,
 <Circuit Diode «PartUsage»>,
 <Motive Force «PartUsage»>,
 <ConnectionUsage([<01256e61-468b-401b-9e89-3f23a2a10f1f «Feature»>] ←→ [<f869a003-9654-404e-9360-f2397b8a16d5 «Feature»>])>]

## Update multiplicities
for `Resistors`, `Diodes`, and `Connections`

In [9]:
multiplicity = circuit_def.ownedMember["Circuit Resistor"].multiplicity
multiplicity.lowerBound._data["value"], multiplicity.upperBound._data["value"] = NUM_RESISTORS

multiplicity = circuit_def.ownedMember["Circuit Diode"].multiplicity
multiplicity.lowerBound._data["value"], multiplicity.upperBound._data["value"] = NUM_DIODES

# Get the ConnectionUsage
for member in circuit_def.ownedMember:
    if member._metatype != "ConnectionUsage":
        continue
    member.multiplicity.lowerBound._data["value"], member.multiplicity.upperBound._data["value"] = NUM_CONNECTORS
    connection = member

## Generate M0 instances from the M1 model

Use the M1 model to start creating a series of instances to represent the circuits that should be analyzed.

In [10]:
m0_interpretations = [
    random_generator_playbook(
        lpg=circuit_lpg,
        name_hints={},
        filtered_feat_packages=[circuit_lpg.model.ownedElement["Circuit Builder"]],
        phase_limit=10,
    ) for _ in range(NUM_INTERPRETATIONS)
]

These edge types are not in the graph: {'ImpliedFeatureTyping'}.


### Sort the interpretations by number of connections
Sorted from `most` to `least`, and pick the first one.

In [11]:
# sort the interpretations from most connections to fewer connections
m0_interpretations = [*sorted(m0_interpretations, key=lambda x: len(x[connection._id]), reverse=True)]
m0_interpretation = m0_interpretations[0]

## Filter M0 Instances for Reasonable Circuits

Until we get more sophisticated and can interpret constraints, the initial approach is to filter out solutions with unanalyzable layouts or trim the layouts to something more tractable.

### Connector End Checks

Look at the ends of the three main kinds of connectors.

In [12]:
p2p = circuit_def.ownedMember["Part to Part"]
p2p.endFeature[0]._id

'4823f832-f7ed-4f37-905b-b5bf4e21bee4'

In [13]:
source_feat, target_feat = p2p.endFeature
for source, target in zip(m0_interpretation[source_feat._id], m0_interpretation[target_feat._id]):
    print(source, "-->", target)

[Circuit#6, Circuit Connection#37, EMF#6, Pin#45] --> [Circuit#6, Circuit Connection#37, Resistor#15, Pin#46]
[Circuit#6, Circuit Connection#38, Resistor#15, Pin#49] --> [Circuit#6, Circuit Connection#38, EMF#6, Pin#48]
[Circuit#6, Circuit Connection#39, Resistor#16, Pin#50] --> [Circuit#6, Circuit Connection#39, Resistor#16, Pin#47]
[Circuit#6, Circuit Connection#40, Resistor#15, Pin#49] --> [Circuit#6, Circuit Connection#40, Resistor#16, Pin#47]
[Circuit#6, Circuit Connection#41, Resistor#16, Pin#50] --> [Circuit#6, Circuit Connection#41, Resistor#15, Pin#46]
[Circuit#6, Circuit Connection#42, EMF#6, Pin#45] --> [Circuit#6, Circuit Connection#42, EMF#6, Pin#48]
[Circuit#6, Circuit Connection#43, Resistor#15, Pin#49] --> [Circuit#6, Circuit Connection#43, EMF#6, Pin#48]
[Circuit#6, Circuit Connection#44, EMF#6, Pin#45] --> [Circuit#6, Circuit Connection#44, EMF#6, Pin#48]
[Circuit#6, Circuit Connection#45, Resistor#15, Pin#49] --> [Circuit#6, Circuit Connection#45, EMF#6, Pin#48]
[Cir

> Janky A.F. connector filter

The cell below filters out:
* self-connections
* duplicate connectors between the same m0 pins

In [42]:
def inspect_graph(interpretation=0):
    unique_connections = get_unique_connections(m0_interpretations[interpretation], p2p.endFeature)
    circuit_stats = {}
    
    def get_el_name(element):
        name = element.name
        name, num = name.split("#")
        name = name if name == "EMF" else name[0]
        return name + num
    
    def legal_predecessors(g, n, valids):
        all_pred = set(g.predecessors(n))
        return {pred for pred in all_pred if pred in valids}
    
    def legal_successors(g, n, valids):
        all_suc = set(g.successors(n))
        return {pred for pred in all_suc if pred in valids}

    graph = nx.DiGraph()
    edges = [
        ((get_el_name(source[-2]), "Pos"), (get_el_name(target[-2]), "Neg"))
        for source, target in unique_connections
    ]
    nodes = {
        n[0]
        for n, _ in edges
        for _, n in edges
    }
    edges += [
        ((node, "Neg"), (node, "Pos"))
        for node in nodes
        if not node.startswith("EMF")
    ]
    graph.add_edges_from(edges)
    
    emf_pos_name = [(node, "Pos") for node in nodes if node.startswith("EMF")][0]
    emf_neg_name = [(node, "Neg") for node in nodes if node.startswith("EMF")][0]
    
    # find all paths in the graph from EMF positive side to EMF negative side
    
    flows = list(nx.all_simple_paths(graph, emf_pos_name, emf_neg_name))
    flow_edges = list(nx.all_simple_edge_paths(graph, emf_pos_name, emf_neg_name))
    
    flowed_nodes = {node for flow in flows for node in flow}
    
    circuit_stats['valid_edges'] = set(sum(flow_edges, []))
    
    valid_graph = nx.DiGraph()
    valid_graph.add_edges_from(circuit_stats['valid_edges'])
    
    # look at junctions in the graph
    
    # split junctions - where a positive pin has multiple outputs
    circuit_stats['split_junctions'] = {
        node: legal_successors(valid_graph, node, flowed_nodes)
        for node in graph.nodes if len(legal_successors(valid_graph, node, flowed_nodes)) > 1
    }
    
    # join junctions - where a negative pin has multiple inputs
    
    circuit_stats['join_junctions'] = {
        node: legal_predecessors(valid_graph, node, flowed_nodes)
        for node in graph.nodes if len(legal_predecessors(valid_graph, node, flowed_nodes)) > 1
    }
    
    circuit_stats['both_ways_junctions'] = {
        node
        for node in graph.nodes if len(legal_successors(valid_graph, node, flowed_nodes)) > 1
        if node in sum(map(list, circuit_stats['join_junctions'].values()), [])
    }
    
    circuit_stats['number_loops'] = len(flows)
    circuit_stats['valid_nodes'] = flowed_nodes
    
    circuit_stats['loop_edges'] = flow_edges
    
    circuit_stats['cleaned_graph'] = valid_graph
    circuit_stats['plain_graph'] = graph
    
    circuit_stats['emf_pins'] = {"+": emf_pos_name, "-": emf_neg_name}
    
    # compute the independent loops on the graph
    
    # EMF positive is always in loop #1
    
    circuit_stats['loop_unique_edges'] = []
    
    if circuit_stats['number_loops'] > 1:
        encountered_nodes = set()
        for indx in range(0, circuit_stats['number_loops']):
            new_edges = []
            for edg in flow_edges[indx]:
                if edg not in encountered_nodes:
                    new_edges.append(edg)
                    encountered_nodes.add(edg)
            circuit_stats['loop_unique_edges'].append(new_edges)
    else:
        circuit_stats['loop_unique_edges'] = list[flow_edges]
    
    return circuit_stats

In [85]:
import ipywidgets as ipyw
import matplotlib.pyplot as plt
from IPython.display import display

def get_unique_connections(instances, feature):
    source_feat, target_feat = feature
    m0_connector_ends = [
        (tuple(source), tuple(target))
        for source, target in zip(instances[source_feat._id], instances[target_feat._id])
    ]

    m0_connector_ends = tuple({
        (source, target)
        for source, target in m0_connector_ends
        if source[:-1] != target[:-1]
    })

    unique_connections = {
        (source[-2:], target[-2:]): (source, target)
        for source, target in m0_connector_ends
    }
    return tuple((source, target) for source, target in unique_connections.values())
    # return tuple((source, target) for source, target in m0_connector_ends)


def draw_circuit(circuit_graph: nx.DiGraph, figsize=None, rad=0.1, arrowsize=40, linewidth=2, layout="kamada_kawai"):
    figsize = figsize or (20, 20)

    color_dict = {
        "blue": "R",
        "#A020F0": "D",
        "red": "EMF",
    }
    layout_algorithm = getattr(nx.layout, f"{layout}_layout")
    node_pos = layout_algorithm(circuit_graph)

    internal_edge_list = [
        (n1, n2)
        for n1, n2 in circuit_graph.edges
        if n1[0] == n2[0]
    ]
    connect_edge_list = [
        edge
        for edge in circuit_graph.edges
        if edge not in internal_edge_list
    ]
    
    nodes = {
        n[0]
        for n, _ in circuit_graph.edges
        for _, n in circuit_graph.edges
    }
    
    emf_pos_name = [(node, "Pos") for node in nodes if node.startswith("EMF")][0]
    emf_neg_name = [(node, "Neg") for node in nodes if node.startswith("EMF")][0]
    
    flow_edges = list(nx.all_simple_edge_paths(circuit_graph, emf_pos_name, emf_neg_name))
    
    loop_edge_list = []
    
    if len(flow_edges) > 1:
        encountered_edges = set(internal_edge_list)
        for indx in range(0, len(flow_edges)):
            new_edges = []
            for edg in flow_edges[indx]:
                if edg not in encountered_edges:
                    new_edges.append(edg)
                    encountered_edges.add(edg)
            loop_edge_list.append(new_edges)
    else:
        loop_edge_list = [connect_edge_list]
        
    loop_edge_tuple = tuple(loop_edge_list)
        
    plt.figure(figsize=figsize)
    for color, colored_nodes in color_dict.items():
        poses = {
            node: loc
            for node, loc in node_pos.items() if node[0].startswith(colored_nodes)
        }
        nx.draw_networkx_nodes(circuit_graph, poses, nodelist=list(poses.keys()), node_size=1200, node_color=color)
    
    color_list_master = ("#f68e11", "#93a109", "#063e00")
    
    # My iterators suck - sorry Santiago
    
    color_list = []
    for indx in range(0, len(flow_edges)):
        color_list.append(color_list_master[indx])
    
    style_tuple = tuple(["-" for _ in range(0, len(flow_edges))])
    color_tuple = tuple(color_list)
    arrow_tuple = tuple(["-|>" for _ in range(0, len(flow_edges))])
    
    #print(arrow_tuple + ("->",))

    #for edgelist, style, edge_color, arrowstyle in zip((connect_edge_list, internal_edge_list), ("-", (0, (5, 5))), ("black", "gray"), ("-|>", "->")):
    for edgelist, style, edge_color, arrowstyle in \
        zip(loop_edge_tuple + tuple([internal_edge_list]), style_tuple + (":",), color_tuple + ("gray",), arrow_tuple + ("->",)):
        nx.draw_networkx_edges(
            circuit_graph,
            node_pos,
            arrowstyle=arrowstyle,
            edgelist=edgelist,
            edge_color=edge_color,
            width=linewidth,
            style=style,
            arrowsize=arrowsize,
            connectionstyle=f"arc3,rad={rad}",
        )

    labels = {(node, polarity): f"{node}{'-' if polarity=='Neg' else '+'}" for node, polarity in node_pos}
    label_options = {"boxstyle": "circle", "ec": "white", "fc": "white", "alpha": 0.0}

    nx.draw_networkx_labels(
        circuit_graph,
        node_pos,
        labels,
        font_size=8,
        font_color="white",
        font_weight="bold",
        bbox=label_options,
    )
    plt.show()

nx_layouts = sorted([
    layout.replace("_layout", "")
    for layout in dir(nx.layout)
    if layout.endswith("_layout")
    and not any(bad_stuff in layout for bad_stuff in ("partite", "rescale", "planar"))
])
    
@ipyw.interact()
def draw_graph(
    interpretation=(0, len(m0_interpretations)-1),
    edge_curvature=(0, 0.5, 0.05),
    linewidth=(1, 3, 0.2),
    layout=nx_layouts,
    clean=True,
):
    try:
        graph = inspect_graph(interpretation)['cleaned_graph' if clean else 'plain_graph']
        draw_circuit(graph, figsize=(20, 10), rad=edge_curvature, linewidth=linewidth, layout=layout)
    except Exception as exc:
        print(f"Interpretation {interpretation} does not form a valid circuit, because: {exc}")

interactive(children=(IntSlider(value=24, description='interpretation', max=49), FloatSlider(value=0.25, descr…

In [63]:
inspect_graph(13)

{'valid_edges': {(('EMF44', 'Pos'), ('R137', 'Neg')),
  (('EMF44', 'Pos'), ('R138', 'Neg')),
  (('R136', 'Neg'), ('R136', 'Pos')),
  (('R136', 'Pos'), ('R138', 'Neg')),
  (('R137', 'Neg'), ('R137', 'Pos')),
  (('R137', 'Pos'), ('R136', 'Neg')),
  (('R138', 'Neg'), ('R138', 'Pos')),
  (('R138', 'Pos'), ('EMF44', 'Neg'))},
 'split_junctions': {('EMF44', 'Pos'): {('R137', 'Neg'), ('R138', 'Neg')}},
 'join_junctions': {('R138', 'Neg'): {('EMF44', 'Pos'), ('R136', 'Pos')}},
 'both_ways_junctions': {('EMF44', 'Pos')},
 'number_loops': 2,
 'valid_nodes': {('EMF44', 'Neg'),
  ('EMF44', 'Pos'),
  ('R136', 'Neg'),
  ('R136', 'Pos'),
  ('R137', 'Neg'),
  ('R137', 'Pos'),
  ('R138', 'Neg'),
  ('R138', 'Pos')},
 'loop_edges': [[(('EMF44', 'Pos'), ('R137', 'Neg')),
   (('R137', 'Neg'), ('R137', 'Pos')),
   (('R137', 'Pos'), ('R136', 'Neg')),
   (('R136', 'Neg'), ('R136', 'Pos')),
   (('R136', 'Pos'), ('R138', 'Neg')),
   (('R138', 'Neg'), ('R138', 'Pos')),
   (('R138', 'Pos'), ('EMF44', 'Neg'))],
  

## Compute Circuit Metrics

Evaluate the circuit for number of branches, see if there are any dead loops or loopbacks in the circuit.

## Determine Number of Unknowns in Circuit

Compute the number of independent voltage and current levels around the circuit.

# OpenMDAO
> Based on OpenMDAO's [nonlinear circuit analysis example](https://openmdao.org/newdocs/versions/latest/examples/circuit_analysis_examples.html).

In [16]:
from importlib import import_module

import networkx as nx
import openmdao.api as om

## Load OpenMDAO component classes

In [17]:
def load_class(class_path: str) -> type:
    *module_path, class_name = class_path.split(".")
    module = import_module(".".join(module_path))
    return getattr(module, class_name)

In [18]:
# TODO: this should be retrieved from the SysML model
components_to_om = {
    "D": "openmdao.test_suite.test_examples.test_circuit_analysis.Diode",
    "R": "openmdao.test_suite.test_examples.test_circuit_analysis.Resistor",
    "Node": "openmdao.test_suite.test_examples.test_circuit_analysis.Node",
}

components_to_om = {
    name: load_class(class_path)
    for name, class_path in components_to_om.items()
}

# FIXME: figure out a way to generalize this patterns
circuit_components = {
    element.name: dict(id=element._id, om=components_to_om[element.name])
    for element in circuit_model.ownedElement["Circuit Builder"].ownedElement
    if element.name in components_to_om
}

## Declare an OpenMDAO Group based on the SysML Model

In [19]:
edges = (
    (("EMF", "Pos"), ("R1", "Neg")),
    (("EMF", "Pos"), ("R2", "Neg")),
    (("R1", "Pos"), ("EMF", "Neg")),
    (("R2", "Pos"), ("D1", "Neg")),
    (("D1", "Pos"), ("EMF", "Neg")),
)
baseline_digraph = nx.DiGraph()
baseline_digraph.add_edges_from(edges)
baseline_data = dict(
    cleaned_graph=baseline_digraph,
    emf_pins={"+": ("EMF", "Pos"), "-": ("EMF", "Neg")}
)
all_params = dict(R1=dict(R=100), R2=dict(R=10000))

In [20]:
class Circuit(om.Group):
    
    def initialize(self):
        self.options.declare("interpretation", types=int)

    def setup(self):
        try:
            interpretation = self.options["interpretation"]
            if RUN_BASELINE:
                data = baseline_data
            else:
                data = inspect_graph(interpretation)
            digraph = data["cleaned_graph"]
        except:
            raise ValueError(f"Interpretation {interpretation} does not produce a valid circuit!")

        V, *_ = next(digraph.successors(data["emf_pins"]["+"]))
        grounded = [el for el, pin in digraph.predecessors(data["emf_pins"]["-"])]
        
        elements = {
            element
            for element, polarity in digraph.nodes
            if not element.upper().startswith("EMF")
        }
        for element in elements:
            comp_cls = components_to_om.get(element[0])
            kwargs = {}
            params = all_params.get(element, {})
            if params:
                print(f"Setting params={params} for {element}")
            if element in grounded:
                kwargs["promotes_inputs"] = [('V_out', 'Vg')]
            if comp_cls is None:
                print(f"{element} doesn't have class!")
                continue
            self.add_subsystem(f"{element}", comp_cls(**params), **kwargs)
            
        connectors = nx.DiGraph()
        connectors.add_edges_from([
            (src, tgt)
            for src, tgt in digraph.edges
            if src[0] != tgt[0]
        ])
        self.node_names = node_names = []
        for node_id, comp in enumerate(nx.connected_components(connectors.to_undirected())):
            node_name = f"node_{node_id}"

            if data["emf_pins"]["-"] in comp:
                print(f"  > Not adding '{node_name}' because it is connected to the ground ({data['emf_pins']['-']})")
                continue
            node_names += [node_name]

            has_pos_emf = data["emf_pins"]["+"] in comp
            if has_pos_emf:
                self.source_node = node_name
            
            n_in = sum(1 for node in comp if node[1] == "Pos")
            n_out = sum(1 for node in comp if node[1] == "Neg")

            kwargs = dict(promotes_inputs=[('I_in:0', 'I_in')]) if has_pos_emf else {}
            node = self.add_subsystem(
                node_name,
                components_to_om["Node"](n_in=n_in, n_out=n_out),
                **kwargs,
            )
            print(f"  > Adding '{node_name}' with {n_in} inputs and {n_out} outputs")
            indeces = {"in": 1*has_pos_emf, "out": 0}
            elec_volt_pins = []
            for element, polarity in comp:
                if element.startswith("EMF"):
                    continue
                elem_dir = "out" if polarity == "Pos" else "in"
                node_dir = "out" if elem_dir == "in" else "in"
                elec_volt_pins += [f"{element}.V_{elem_dir}"]

                node_current = f"{node_name}.I_{node_dir}:{indeces[node_dir]}"
                self.connect(f"{element}.I", node_current)
                print(f"  > Connecting currents: {element}.I --> {node_current}")
                indeces[node_dir] += 1
            if elec_volt_pins:
                try:
                    self.connect(f"{node_name}.V", elec_volt_pins)
                    print(f" >  Connecting voltages for node {node_name}.V --> {elec_volt_pins}")
                except:
                    print(f"  ! Could not connect: {node_name}.V --> {elec_volt_pins}")

        self.nonlinear_solver = om.NewtonSolver()
        self.linear_solver = om.DirectSolver()

        self.nonlinear_solver.options['iprint'] = 2
        self.nonlinear_solver.options['maxiter'] = 10
        self.nonlinear_solver.options['solve_subsystems'] = True
        self.nonlinear_solver.linesearch = om.ArmijoGoldsteinLS()
        self.nonlinear_solver.linesearch.options['maxiter'] = 10
        self.nonlinear_solver.linesearch.options['iprint'] = 2
        
        return True

In [21]:
import traceback

# RUN_BASELINE = False

for idx in range(len(m0_interpretations)):
    try:
        print(f">>> Trying to setup {idx}!")
        p = om.Problem()
        model = p.model
        circuit_name = f'circuit_{idx}'
        model.add_subsystem(circuit_name, Circuit(interpretation=idx))
        is_valid = p.setup()
        if not is_valid:
            raise ValueError(f"  ! Interpretation {idx} does not produce a valid circuit!")
        print(f" >> Successfully set up {idx}")
        try:
            circuit = getattr(p.model, circuit_name)
            p.set_val(f"{circuit_name}.I_in", 0.1)
            p.set_val(f"{circuit_name}.Vg", 0)
            for node_name in circuit.node_names:
                p.set_val(
                    f"{circuit_name}.{node_name}.V",
                    (10. if node_name == circuit.source_node else 0.1)
                )
            p.run_model()
            print(f"  + Successfully ran {idx}!\n")
        except Exception as exc:
            print(f"  ! Failed to run {idx}!")
    except Exception as exc:
        print(f"  ! Failed to setup {idx}!")
        # print(traceback.format_exc())
    if RUN_BASELINE:
        break

>>> Trying to setup 0!
Setting params={'R': 10000} for R2
Setting params={'R': 100} for R1
  > Adding 'node_0' with 1 inputs and 2 outputs
  > Connecting currents: R1.I --> node_0.I_out:0
  > Connecting currents: R2.I --> node_0.I_out:1
 >  Connecting voltages for node node_0.V --> ['R1.V_in', 'R2.V_in']
  > Not adding 'node_1' because it is connected to the ground (('EMF', 'Neg'))
  > Adding 'node_2' with 1 inputs and 1 outputs
  > Connecting currents: R2.I --> node_2.I_in:0
  > Connecting currents: D1.I --> node_2.I_out:0
 >  Connecting voltages for node node_2.V --> ['R2.V_out', 'D1.V_in']
 >> Successfully set up 0

circuit_0
NL: Newton 0 ; 0.00140007143 1
|  LS: AG 1 ; 6.9706756e+152 1
|  LS: AG 2 ; 5.7657237e+69 0.5
|  LS: AG 3 ; 1.65822441e+28 0.25
|  LS: AG 4 ; 28121471.5 0.125
|  LS: AG 5 ; 0.000956185889 0.0625
NL: Newton 1 ; 0.000956185889 0.682955077
NL: Newton 2 ; 2.29803213e-05 0.0164136778
NL: Newton 3 ; 2.76843426e-07 0.000197735216
NL: Newton 4 ; 4.67602463e-11 3.339847

In [22]:
p.model.list_inputs()

11 Input(s) in 'model'

varname      val         
-----------  ------------
circuit_0
  R2
    V_in     [9.90804735]
    V_out    [0.71278185]
  D1
    V_in     [0.71278185]
    V_out    [0.]        
  R1
    V_in     [9.90804735]
    V_out    [0.]        
  node_0
    I_in:0   [0.1]       
    I_out:0  [0.09908047]
    I_out:1  [0.00091953]
  node_2
    I_in:0   [0.00091953]
    I_out:0  [0.00091953]




[('circuit_0.R2.V_in', {'val': array([9.90804735])}),
 ('circuit_0.R2.V_out', {'val': array([0.71278185])}),
 ('circuit_0.D1.V_in', {'val': array([0.71278185])}),
 ('circuit_0.D1.V_out', {'val': array([0.])}),
 ('circuit_0.R1.V_out', {'val': array([0.])}),
 ('circuit_0.R1.V_in', {'val': array([9.90804735])}),
 ('circuit_0.node_0.I_in:0', {'val': array([0.1])}),
 ('circuit_0.node_0.I_out:0', {'val': array([0.09908047])}),
 ('circuit_0.node_0.I_out:1', {'val': array([0.00091953])}),
 ('circuit_0.node_2.I_in:0', {'val': array([0.00091953])}),
 ('circuit_0.node_2.I_out:0', {'val': array([0.00091953])})]

In [23]:
print(idx)
om.view_connections(p)

0


In [24]:
om.n2(p)